In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from functools import partial
from sklearn.externals import joblib
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import LinearRegression

sys.path.append('../')
from src.utils import parallel_apply
from src.feature_extraction import add_features_in_group

DIR = '/PATH/TO/YOUR/DATA'
description = pd.read_csv(('HomeCredit_columns_description.csv'),encoding = 'latin1')
application = pd.read_csv('application_train.csv')
bureau = pd.read_csv('bureau.csv')
bureau_balance = pd.read_csv('bureau_balance.csv')

In [2]:
from IPython.display import display

In [3]:
bureau_balance.shape

(27299925, 3)

In [4]:
display(bureau_balance.head(2))

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C


In [5]:
bureau_balance = bureau_balance.merge(bureau[['SK_ID_CURR', 'SK_ID_BUREAU']], on='SK_ID_BUREAU', how='right')
bureau_balance.shape

(25121815, 4)

In [6]:
display(bureau_balance.head())

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS,SK_ID_CURR
0,5715448,0.0,C,380361
1,5715448,-1.0,C,380361
2,5715448,-2.0,C,380361
3,5715448,-3.0,C,380361
4,5715448,-4.0,C,380361


In [7]:
# bureau_balance['STATUS'].value_counts

In [8]:
def _status_to_int(status):
    if status in ['X', 'C']:
        return 0
    if pd.isnull(status):
        return np.nan
    return int(status)

In [9]:
bureau_balance['bureau_balance_dpd_level'] = bureau_balance['STATUS'].apply(_status_to_int)
bureau_balance['bureau_balance_status_unknown'] = (bureau_balance['STATUS'] == 'X').astype(int)

In [10]:
display(bureau_balance.tail(15),bureau_balance.head(15))

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS,SK_ID_CURR,bureau_balance_dpd_level,bureau_balance_status_unknown
25121800,5057665,NaN,NaN,288038,NaN,0
25121801,5057676,NaN,NaN,445222,NaN,0
25121802,5057684,NaN,NaN,317217,NaN,0
25121803,5057695,NaN,NaN,433007,NaN,0
25121804,5057703,NaN,NaN,433007,NaN,0
25121805,5057708,NaN,NaN,433007,NaN,0
25121806,5057718,NaN,NaN,352790,NaN,0
25121807,5057725,NaN,NaN,352790,NaN,0
25121808,5057734,NaN,NaN,375755,NaN,0
25121809,5057742,NaN,NaN,375755,NaN,0


,SK_ID_BUREAU,MONTHS_BALANCE,STATUS,SK_ID_CURR,bureau_balance_dpd_level,bureau_balance_status_unknown
0,5715448,0.0,C,380361,0.0,0
1,5715448,-1.0,C,380361,0.0,0
2,5715448,-2.0,C,380361,0.0,0
3,5715448,-3.0,C,380361,0.0,0
4,5715448,-4.0,C,380361,0.0,0
5,5715448,-5.0,C,380361,0.0,0
6,5715448,-6.0,C,380361,0.0,0
7,5715448,-7.0,C,380361,0.0,0
8,5715448,-8.0,C,380361,0.0,0
9,5715448,-9.0,0,380361,0.0,0


In [11]:
groupby = bureau_balance.groupby(['SK_ID_CURR'])
features = pd.DataFrame({'SK_ID_CURR': bureau_balance['SK_ID_CURR'].unique()})




In [15]:
bureau_balance.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS,SK_ID_CURR,bureau_balance_dpd_level,bureau_balance_status_unknown
0,5715448,0.0,C,380361,0.0,0
1,5715448,-1.0,C,380361,0.0,0
2,5715448,-2.0,C,380361,0.0,0
3,5715448,-3.0,C,380361,0.0,0
4,5715448,-4.0,C,380361,0.0,0


In [12]:
features.head(10)

,SK_ID_CURR
0,380361
1,401081
2,448798
3,400434
4,393321
5,357810
6,370938
7,386362
8,265640
9,318566


In [13]:
# def last_k_installment_features(gr, periods):
#     gr_ = gr.copy()

#     features = {}
#     for period in periods:
#         if period > 10e10:
#             period_name = 'all_installment_'
#             gr_period = gr_.copy()
#         else:
#             period_name = 'last_{}_'.format(period)
#             gr_period = gr_[gr_['MONTHS_BALANCE'] >= (-1) * period]

#         features = add_features_in_group(features, gr_period, 'bureau_balance_dpd_level',
#                                              ['sum', 'mean', 'max', 'std', 'skew', 'kurt'],
#                                              period_name)
#         features = add_features_in_group(features, gr_period, 'bureau_balance_status_unknown',
#                                              ['sum', 'mean'],
#                                              period_name)
#     return features

In [14]:
# func = partial(last_k_installment_features, periods=[6, 12, 24, 60, 10e16])
# g = parallel_apply(groupby, func, index_name='SK_ID_CURR', num_workers=10, chunk_size=10000).reset_index()
# features = features.merge(g, on='SK_ID_CURR', how='left')